In [ ]:
import argparse, os, sys, numpy as np
import open3d as o3d
import trimesh

def load_any_ply(path):
    # Try as triangle mesh first
    m = o3d.io.read_triangle_mesh(path)
    if m.has_triangles():
        m.compute_vertex_normals()
        return m, "mesh"
    # Fallback: point cloud
    p = o3d.io.read_point_cloud(path)
    if len(p.points) == 0:
        raise RuntimeError("No geometry found in PLY.")
    return p, "points"

def estimate_normals_if_needed(pcd, radius=0.05, max_nn=40):
    if not pcd.has_normals():
        pcd.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn)
        )
        # Consistent orientation improves Poisson
        pcd.orient_normals_consistent_tangent_plane(20)

def reconstruct_mesh_from_points(
    pcd,
    method="poisson",
    voxel_ds=0.015,
    poisson_depth=10,
    poisson_scale=1.1,
    bp_radii=(0.02, 0.04, 0.08)
):
    # Downsample for speed (tune to your scene scale in meters)
    if voxel_ds and voxel_ds > 0:
        pcd = pcd.voxel_down_sample(voxel_ds)

    estimate_normals_if_needed(pcd)

    if method == "poisson":
        mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd, depth=poisson_depth, scale=poisson_scale, linear_fit=False
        )
        # Trim to the input bounds to remove the "balloon"
        aabb = pcd.get_axis_aligned_bounding_box()
        mesh = mesh.crop(aabb)
    elif method == "ballpivot":
        # Ball pivoting expects a radius list ~ feature sizes
        radii = o3d.utility.DoubleVector(list(bp_radii))
        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
            pcd, radii
        )
    else:
        raise ValueError("method must be 'poisson' or 'ballpivot'")

    mesh.remove_unreferenced_vertices()
    mesh.compute_vertex_normals()
    return mesh

def remove_small_components(mesh, keep="largest", min_triangles=500):
    labels, _ = mesh.cluster_connected_triangles()
    labels = np.asarray(labels)
    counts = np.bincount(labels)
    if keep == "largest":
        keep_label = np.argmax(counts)
        tri_mask = labels == keep_label
        mesh = mesh.select_by_index(np.where(tri_mask)[0], prefer_triangles=True)
    else:
        # Drop tiny islands below threshold
        keep_labels = np.where(counts >= min_triangles)[0]
        tri_mask = np.isin(labels, keep_labels)
        mesh = mesh.select_by_index(np.where(tri_mask)[0], prefer_triangles=True)
    mesh.remove_unreferenced_vertices()
    return mesh

def decimate(mesh, target_tris):
    mesh = mesh.filter_smooth_simple(number_of_iterations=1)
    mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=int(target_tris))
    mesh.compute_vertex_normals()
    return mesh

def export_glb(mesh, out_path):
    # Convert Open3D mesh -> Trimesh and write GLB
    verts = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)
    tm = trimesh.Trimesh(vertices=verts, faces=faces, process=False)
    # Ensure y-up glTF (Blender/three default); Open3D uses right-handed coords already.
    data = trimesh.exchange.gltf.export_glb(tm)
    with open(out_path, "wb") as f:
        f.write(data)

def main():
    ap = argparse.ArgumentParser(description="PLY (points or mesh) -> GLB mesh")
    ap.add_argument("--input", "-i", required=True, help="Input .ply")
    ap.add_argument("--output", "-o", default=None, help="Output .glb (default: input name)")
    ap.add_argument("--method", choices=["poisson","ballpivot"], default="poisson",
                    help="Surface reconstruction method for point clouds")
    ap.add_argument("--voxel", type=float, default=0.015, help="Voxel downsample size (m)")
    ap.add_argument("--depth", type=int, default=10, help="Poisson octree depth (detail)")
    ap.add_argument("--scale", type=float, default=1.1, help="Poisson 'scale' (balloon factor)")
    ap.add_argument("--bp_radii", type=float, nargs="+", default=[0.02,0.04,0.08],
                    help="Ball-pivot radii (m), space-separated")
    ap.add_argument("--target_tris", type=int, default=150_000, help="Triangle budget after decimation")
    ap.add_argument("--keep_mode", choices=["largest","threshold"], default="largest",
                    help="Island cleanup strategy")
    ap.add_argument("--min_triangles", type=int, default=500, help="Used if keep_mode=threshold")
    args = ap.parse_args()

    src = args.input
    out = args.output or os.path.splitext(src)[0] + ".glb"

    geom, gtype = load_any_ply(src)

    if gtype == "mesh":
        mesh = geom
        if len(mesh.triangles) == 0:
            # Degenerate mesh that only has vertices -> treat as points
            pcd = o3d.geometry.PointCloud(mesh.vertices)
            mesh = reconstruct_mesh_from_points(
                pcd, method=args.method, voxel_ds=args.voxel,
                poisson_depth=args.depth, poisson_scale=args.scale,
                bp_radii=tuple(args.bp_radii)
            )
        # Clean + decimate
        mesh = remove_small_components(mesh, keep=args.keep_mode, min_triangles=args.min_triangles)
        if args.target_tris and len(mesh.triangles) > args.target_tris:
            mesh = decimate(mesh, args.target_tris)
        export_glb(mesh, out)
    else:
        # points -> mesh -> clean -> decimate -> glb
        mesh = reconstruct_mesh_from_points(
            geom, method=args.method, voxel_ds=args.voxel,
            poisson_depth=args.depth, poisson_scale=args.scale,
            bp_radii=tuple(args.bp_radii)
        )
        mesh = remove_small_components(mesh, keep=args.keep_mode, min_triangles=args.min_triangles)
        if args.target_tris and len(mesh.triangles) > args.target_tris:
            mesh = decimate(mesh, args.target_tris)
        export_glb(mesh, out)

    print(f"✅ Wrote {out}")

if __name__ == "__main__":
    main()